In [ ]:
import os
import certifi
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter

# Set the SSL certificate file
os.environ['SSL_CERT_FILE'] = certifi.where()
city = "INSIRA NOME DA CIDADE AQUI"
country = "INSIRA NOME DO PAÍS AQUI"

# Make sure the API key is set in the environment variables
api_key = os.environ.get('API_KEY')
# Load your dataset
file_path = 'data/cleaned/greater-vancouver-area-cleaned.csv'
data = pd.read_csv(file_path)

# Initialize the geocoder with the API key
geolocator = GoogleV3(api_key=api_key)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to get coordinates
def get_coordinates(intersection, city, country):
    try:
        location = geocode(intersection + ", " + city + ", " + country)
        return (location.latitude, location.longitude) if location else (None, None)
    except Exception as e:
        print(f"Error geocoding {intersection}: {e}")
        return (None, None)

# Apply the function to the nearest intersection column
data[['latitude', 'longitude']] = data['Nearest intersection'].apply(lambda x: pd.Series(get_coordinates(x)))

# Save the updated dataframe with coordinates to a new CSV file
output_file_path = f'data/cleaned/coordinates/{city}-{country}-area-with-coordinates.csv'
data.to_csv(output_file_path, index=False)

# Display the first few rows to check the results
print(data.head())
